# Chapter 2 Image classification

## Training MNIST model in Keras

### Preparing the dataset


In [1]:
import keras

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 128 
no_classes = 10
epochs = 2
image_height, image_width = 28,28

In [3]:
## Loading the dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
### Formatting the data to make each pixel and individual array adding that way
###  other dimension to the data
x_train = x_train.reshape(x_train.shape[0], image_height, image_width, 1)

x_test = x_test.reshape(x_test.shape[0], image_height, image_width, 1)
input_shape = (image_height, image_width, 1)

In [5]:
### Converting integers from 0 to 255 to float allowing division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


In [6]:
### normalizing data to be from 0 to 1 to make it easier to train
x_train /= 255
x_test /= 255

In [7]:
#print(x_train[0][12])
print(y_test)

[7 2 1 ... 4 5 6]


In [8]:
### making one shoot encoding classes
y_train = keras.utils.to_categorical(y_train, no_classes)
y_test = keras.utils.to_categorical(y_test, no_classes)

In [9]:
def simple_cnn(input_shape):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters = 64,
                                  kernel_size= (3,3),
                                 activation= 'relu',
                                 input_shape = input_shape)
             )
    model.add(keras.layers.Conv2D(filters = 128,
                                 kernel_size= (3,3),
                                  activation = 'relu'
                                 )
             )
    model.add( keras.layers.MaxPooling2D(pool_size = (2,2) ) )
    model.add( keras.layers.Dropout(rate = 0.3))
    model.add( keras.layers.Flatten()) ##Make it one dimension
    model.add( keras.layers.Dense(units= 1024, activation= 'relu'))
    model.add( keras.layers.Dropout(rate= 0.3) )
    model.add( keras.layers.Dense(units= no_classes, activation='softmax'))
    model.compile(loss= keras.losses.categorical_crossentropy,
                  optimizer = keras.optimizers.Adam(),
                  metrics = ['accuracy']
                )
    return model

simple_cnn_model = simple_cnn(input_shape)
    

In [ ]:
simple_cnn_model.fit(x_train, y_train, batch_size, epochs, (x_test, y_test))

train_loss, train_accuracy  = simple_cnn_model.evaluate(x_train, y_train,
                                                        verbose= 0)
print("Train data loss:" , train_loss)
print("train data accuracy:", train_accuracy)

Epoch 1/2
Epoch 2/2


In [ ]:
test_loss, test_accuracy = simple_cnn_model.evaluate(x_test, y_test, verbose= 0)

print("Test data loss:" , test_loss)
print("Test data accuracy:", test_accuracy)

## Training model cats vs dogs

### Preparing data

In [10]:
import os
import shutil

In [11]:
work_dir = '../data/'

In [16]:
#print(os.listdir(os.path.join(work_dir, 'train')))
image_names = sorted(os.listdir(os.path.join(work_dir, 'train')))
print(image_names)

['.DS_Store', 'cat', 'dog']


In [17]:
def copy_files(prefix_str, range_start, range_end, target_dir):
    image_paths = [os.path.join(work_dir, 'train', prefix_str + '.' 
                                + str(i) + '.jpg' )
                  for i in range(range_start, range_end)]
    dest_dir = os.path.join(work_dir, 'data', target_dir, prefix_str)
    os.makedirs(dest_dir)
    for image_path in image_paths:
        shutil.copy(image_path, dest_dir)

In [19]:
## copy files with subfolders for dogs and cats
def copy_files2(prefix_str, range_start, range_end, target_dir):
    image_paths = [os.path.join(work_dir, 'train/'+ prefix_str, prefix_str + '.' 
                                + str(i) + '.jpg' )
                  for i in range(range_start, range_end)]
    dest_dir = os.path.join(work_dir, 'data', target_dir, prefix_str)
    os.makedirs(dest_dir)
    for image_path in image_paths:
        shutil.copy(image_path, dest_dir)

In [21]:
##alternative with different folder setup
#copy_files2('dog', 0, 800, 'train')
#copy_files2('cat', 0, 800, 'train')
copy_files2('dog', 800, 1000, 'test')
copy_files2('cat', 800, 1000, 'test')

In [29]:
print(work_dir)

../data/


In [30]:
image_height, image_width = 150,150
train_dir = os.path.join(work_dir, "data", 'train')
test_dir = os.path.join(work_dir, "data", 'test')
no_classes = 2
no_validation = 400
epochs = 2
batch_size = 200
no_train = 1600
no_test = 400
input_shape = (image_height, image_width, 3)
epoch_steps = no_train // batch_size
test_steps = no_test // batch_size
print(epoch_steps)
print(test_steps)

8
2


In [31]:
simple_cnn_model = simple_cnn(input_shape)

In [32]:
generator_train = keras.preprocessing.image.ImageDataGenerator(rescale= 1. / 255)
generator_test = keras.preprocessing.image.ImageDataGenerator(rescale = 1. /255)


In [33]:
train_images = generator_train.flow_from_directory(train_dir,
                                                   batch_size = batch_size,
                                                   target_size = (image_width, image_height)
                                                  )
test_images = generator_test.flow_from_directory(test_dir,
                                                 batch_size=batch_size,
                                                 target_size = (image_width, image_height)
                                                )

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
simple_cnn_model.fit_generator(train_images,
                               steps_per_epoch= epoch_steps,
                               epochs = epochs,
                               validation_data = test_images,
                               validation_steps = test_steps
                              )

Epoch 1/2


### Augmenting the dataset